# Tutorial - Settlement calculations with ```groundhog```

```groundhog``` contains functionality to analyse the settlement of foundations under applied loads. Elastic stress solutions are available for strip foundation, circular foundations and rectangular foundations.

Moreover, the basic functionality of ```groundhog``` for working with soil profiles and rapidly deriving correlations can be illustrated in this tutorial.

In this example, we will derive the compression index $ C_c $ and the recompression index $ C_r $ based on measured unit weights for a saturated cohesive soil. We will use these values to calculate the primary consolidation settlement beneath a strip footing, circular footing and rectangular footing.

## Library imports

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

## Selection of soil parameters

### Loading unit weight data

Two data files are provided to specify a distribution of unit weight vs depth. Bulk unit weight $ \gamma $ has been derived from a volume mass calculation and has also been derived from water content (for a saturated soil) using the following relation:

$$ \gamma = \left( \frac{G_s \cdot (1 + w)}{1 + w \cdot G_s} \right) \cdot \gamma_w $$

In [ ]:
uw_data = pd.read_excel("Data/demo_settlement_uw.xlsx")

We can plot this data vs depth. We can see there is a reasonable amount of scatter in the first 0.5m.

In [ ]:
import plotly.express as px
fig = px.scatter(uw_data, x='Bulk unit weight [kN/m3]', y='z [m]', color='Method')
fig['layout']['xaxis1'].update(title='Bulk unit weight [kN/m3]', side='top', anchor='y', range=(12, 20))
fig['layout']['yaxis1'].update(title='Depth below mudline [m]', autorange='reversed')
fig.show()

### Definition of a soil profile and bulk unit weight selection

```groundhog``` allows soil profile manipulations through the ```SoilProfile``` object. We can create a ```SoilProfile``` for soil parameter selection.

Initially, we will model a profile with 0.5m thick layers.

In [ ]:
from groundhog.general.soilprofile import SoilProfile

We can define the soil profile from a Python dictionary containing an array with the depths of the top of each layer, an array with the bottom of each layer and an array with the soil type of each layer.

In [ ]:
sp = SoilProfile({
    "Depth from [m]": [0, 0.5, 1, 1.5, 2, 2.5, 3],
    "Depth to [m]": [0.5, 1, 1.5, 2, 2.5, 3, 3.5],
    "Soil type": ["CLAY", "CLAY", "CLAY", "CLAY", "CLAY", "CLAY","CLAY"]
})
sp

The ```SoilProfile``` object has a ```selection_soilparameter``` which allows an automatic first eastimate of soil parameters in a profile. 

We will make an estimate based on the test data provided. By default, the average trend will be selected.

In [ ]:
sp.selection_soilparameter(
    'Total unit weight [kN/m3]', depths=uw_data['z [m]'], values=uw_data['Bulk unit weight [kN/m3]'])

The selection provided by the software can be used as an initial guess and the engineer can then modify the selected values.

The choice of the program looks reasonable in the first layer. However, as this is normally consolidated clay, decreases of unit weight with depth are unlikely. So we will amend the first guess.

In [ ]:
uw_fig = sp.plot_profile(
    (('Total unit weight [kN/m3]',),),
    xranges=((12,20),),
    xtitles=('Total unit weight [kN/m3]',),
    showfig=False)
uw_fig.add_trace(
    go.Scatter(x=uw_data['Bulk unit weight [kN/m3]'], y=uw_data['z [m]'], mode='markers', name='data'),
    row=1, col=2)
uw_fig.show()

We can modify the unit weight in the layers. The syntax for this is Pandas syntax. Several tutorials are available online to start working with Pandas. The code below modifies the unit weight in each layer by specifying a new array of unit weights. With every layer, we let the unit weight increase by 0.25kN/m$^3$.

In [ ]:
sp['Total unit weight [kN/m3]'] = [16, 16.25, 16.5, 16.75, 17, 17.25, 17.5]

We can replot the selection. This shows that this is a reasonable selection.

In [ ]:
uw_fig = sp.plot_profile(
    (('Total unit weight [kN/m3]',),),
    xranges=((12,20),),
    xtitles=('Unit weight [kN/m3]',),
    showfig=False)
uw_fig.add_trace(
    go.Scatter(x=uw_data['Bulk unit weight [kN/m3]'], y=uw_data['z [m]'], mode='markers', name='data'),
    row=1, col=2)
uw_fig.show()

### Calculation of overburden pressure

Based on the definition of bulk unit weight, the stress distribution vs depth can be plotted. This is done using the method ```calculate_overburden``` of the ```SoilProfile``` object. By default, the waterlevel is at soil surface and the soil is fully saturated.

The routine will calculate total vertical stress, vertical effective stress and hydrostatic pressure.

In [ ]:
sp.calculate_overburden()
sp

We can plot the overburden pressure vs depth.

In [ ]:
stress_fig = sp.plot_profile(
    (('Total unit weight [kN/m3]',),
     ('Vertical total stress [kPa]', 'Vertical effective stress [kPa]', 'Hydrostatic pressure [kPa]'),),
    showlegends=((False,), (True, True, True)),
    xranges=((12,20), (0, 60)),
    xtitles=('Unit weight [kN/m3]', 'Vertical stress [kPa]'))

### Calculation of initial void ratio and water content

The initial void ratio and water content can be derived from the bulk unit weight using the function ```voidratio_bulkunitweight``` in ```groundhog```. This function calculates the void ratio and water content from bulk unit weight for a saturated soil using the following formulae.

$$ \gamma = \left( \frac{G_s + S e}{1 + e} \right) \gamma_w $$

$$ \implies e = \frac{\gamma_w G_s - \gamma}{\gamma - S \gamma_w} $$

$$ w = \frac{S e}{G_s} $$

```SoilProfile``` objects in ```groundhog``` have a method ```applyfunction``` which can apply any function to the rows of a soil profile. The parameters of the function are mapped to soil parameters in the dictionary ```parametermapping```.

We can apply this function twice, once for the calculation of void ratio (result key ```'e [-]'``` in the result of ```voidratio_bulkunitweight```) and once for the calculation of water content (result key ```'w [-]'``` in the result of ```voidratio_bulkunitweight```).

We can print the resulting soil profile to the notebook.

In [ ]:
from groundhog.siteinvestigation.classification.phaserelations import voidratio_bulkunitweight

In [ ]:
sp.applyfunction(
    function=voidratio_bulkunitweight,
    outputkey="Void ratio [-]", resultkey="e [-]",
    parametermapping={
        'bulkunitweight': "Total unit weight [kN/m3]"
    })
sp.applyfunction(
    function=voidratio_bulkunitweight,
    outputkey="Water content [-]", resultkey="w [-]",
    parametermapping={
        'bulkunitweight': "Total unit weight [kN/m3]"
    })
sp

The initial void ratio is required to calculate the settlements.

### Selection of $ C_c $ and $ C_r $

The compression index $ C_c $ and recompression index $ C_r $ describe the slope of the virgin compression line and the recompression line respectively in $ e - \log_{10} ( \sigma_v^{\prime} ) $ space:

$$ C_c = - \frac{e_2 - e_1}{\log_{10} \left( \frac{\sigma^{\prime}_{v,2}}{\sigma^{\prime}_{v,1}} \right)} \quad \text{Virgin compression line} \\ C_r = - \frac{e_2 - e_1}{\log_{10} \left( \frac{\sigma^{\prime}_{v,2}}{\sigma^{\prime}_{v,1}} \right)} \quad \text{Recompression line}
$$

Several correlations exist which correlate the compression and recompression indices with water content, void ratio, ...

```groundhog``` implements the correlation ```compressionindex_watercontent_koppula``` which is based on a study by Koppula (1981) which found a direct relation between the compression index $ C_c $ and the natural water content $ w $. The compression index is between 5 and 10 times higher than the recompression index $ C_r $. Here, we assume a ratio of 7.5 between both indices.

We can again determine the indices using the ```applyfunction``` method on the ```SoilProfile``` object.

In [ ]:
from groundhog.siteinvestigation.correlations.cohesive import compressionindex_watercontent_koppula

In [ ]:
sp.applyfunction(
    function=compressionindex_watercontent_koppula,
    outputkey="Cc [-]", resultkey="Cc [-]",
    parametermapping={
        'water_content': "Water content [-]"
    })
sp.applyfunction(
    function=compressionindex_watercontent_koppula,
    outputkey="Cr [-]", resultkey="Cr [-]",
    parametermapping={
        'water_content': "Water content [-]"
    })
sp

Note that this choice of indices should be benchmarked against or replaced by values obtained from oedometer tests.

## Calculation of vertical effective stress increases

The vertical effective stress increases for elastic halfspaces can be obtained using the formulae derived by Boussinesq.

```groundhog``` implements the calculation of vertical effective stress increases below strip footings, the center of circular footings and the corner of a rectangular footing. The module ```groundhog.shallowfoundations.stressdistribution``` contains the necessary functions.

In [ ]:
from groundhog.shallowfoundations.stressdistribution import stresses_stripload, stresses_circle, stresses_rectangle

### Illustration of stress increase functions

We can calculate the stresses below each of the foundations using the relevant functions. To illustrate the functions, we will first use a separate soil profile with a fine grid.

Note that NumPy sometimes uses number approximations (e.g. 0.099999999 instead of 0.1) which could cause the layer continuity check to fail. For this reason, we round the NumPy arrays.

The code below creates a soilprofile with 0.01m thick layers.

In [ ]:
sp_fine = SoilProfile({
    "Depth from [m]": np.round(np.linspace(0, 2.99, 300), 2),
    "Depth to [m]": np.round(np.linspace(0.01, 3, 300), 2)
})

For the application of the stress formulae, we need to define the depth at the layer center. The ```SoilProfile``` object has a method ```calculate_center``` which does this and adds a column ```Depth center [m]``` to the soil profile.

In [ ]:
sp_fine.calculate_center()
sp_fine.head()

We can now calculate the vertical effective stress for each of the cases using the ```applyfunction``` method. The documentation of ```groundhog``` contains all information on the Boussinesq functions.

We will calculate the stress increase below:

   - The center of a 0.5m wide strip footing (superposition stress increase below corner of 2 x 0.25m strip footings)
   - The center of a circular footing with a diameter of 0.5m
   - The center of a square footing width 0.5m long edges (superposition of stress increase below the corner of 4 x square footing with 0.125m edges

In [ ]:
sp_fine.applyfunction(
    function=stresses_stripload,
    outputkey="delta sigma z half strip [kPa]", resultkey="delta sigma z [kPa]",
    parametermapping={
        'z': "Depth center [m]"
    },
    x=0.25,
    width=0.25,
    imposedforce=10)
sp_fine["delta sigma z strip [kPa]"] = 2 * sp_fine["delta sigma z half strip [kPa]"]
sp_fine.applyfunction(
    function=stresses_circle,
    outputkey="delta sigma z circle [kPa]", resultkey="delta sigma z [kPa]",
    parametermapping={
        'z': "Depth center [m]"
    },
    footing_radius=0.25,
    imposedstress=10,
    poissonsratio=0.495)
sp_fine.applyfunction(
    function=stresses_rectangle,
    outputkey="delta sigma z quarter rectangle [kPa]", resultkey="delta sigma z [kPa]",
    parametermapping={
        'z': "Depth center [m]"
    },
    imposedstress=10,
    length=0.125,
    width=0.125)
sp_fine["delta sigma z rectangle [kPa]"] = 4 * sp_fine["delta sigma z quarter rectangle [kPa]"]

We can plot these stresses vs depth:

In [ ]:
stressfig = go.Figure()
stressfig.add_trace(
    go.Scatter(x=sp_fine['delta sigma z strip [kPa]'],
               y=sp_fine['Depth center [m]'], mode='lines', name='Strip'))
stressfig.add_trace(
    go.Scatter(x=sp_fine['delta sigma z circle [kPa]'],
               y=sp_fine['Depth center [m]'], mode='lines', name='Circle'))
stressfig.add_trace(
    go.Scatter(x=sp_fine['delta sigma z rectangle [kPa]'],
               y=sp_fine['Depth center [m]'], mode='lines', name='Rectangle'))
stressfig['layout']['xaxis1'].update(title='Stress increase [kPa]')
stressfig['layout']['yaxis1'].update(title='Depth [m]', autorange='reversed')
stressfig.show()

### Application of stress increase functions

We can now apply these functions to the soil profile we had already established. We will calculate the vertical stress increase at the center of each layer.

We will also calculate the layer thickness using the ```calculate_layerthickness``` method.

In [ ]:
sp.calculate_center()
sp.calculate_layerthickness()

In [ ]:
sp.applyfunction(
    function=stresses_stripload,
    outputkey="delta sigma z half strip [kPa]", resultkey="delta sigma z [kPa]",
    parametermapping={
        'z': "Depth center [m]"
    },
    x=0.25,
    width=0.25,
    imposedforce=10)
sp["delta sigma z strip [kPa]"] = 2 * sp["delta sigma z half strip [kPa]"]
sp.applyfunction(
    function=stresses_circle,
    outputkey="delta sigma z circle [kPa]", resultkey="delta sigma z [kPa]",
    parametermapping={
        'z': "Depth center [m]"
    },
    footing_radius=0.25,
    imposedstress=10,
    poissonsratio=0.495)
sp.applyfunction(
    function=stresses_rectangle,
    outputkey="delta sigma z quarter rectangle [kPa]", resultkey="delta sigma z [kPa]",
    parametermapping={
        'z': "Depth center [m]"
    },
    imposedstress=10,
    length=0.125,
    width=0.125)
sp["delta sigma z rectangle [kPa]"] = 4 * sp["delta sigma z quarter rectangle [kPa]"]

We can cross-check the calculated stress against the values plotted above. This shows that the stress increase reduces quite rapidly with depth, especially for the rectangular case.

In [ ]:
sp

## Calculation of settlement

The unit weights suggest a normally consolidated soil, we can therefore use the function ```primaryconsolidationsettlement_nc``` to calculate the settlement according to the following formula:

$$ \Delta z = \frac{H_0}{1 + e_0} C_c \log_{10} \frac{\sigma_{v0}^{\prime} + \Delta \sigma_v^{\prime}}{\sigma_{v0}^{\prime}} $$

In [ ]:
from groundhog.shallowfoundations.settlement import primaryconsolidationsettlement_nc

The only remaining operation before we can apply the formula is to calculate the initial vertical effective stress at the center of the layer. We can do this by averaging the vertical effective stress at top and bottom of the layer using the ```calculate_parameter_center``` method using ```Effective vertical stress [kPa]``` as the parameter.

In [ ]:
sp.calculate_parameter_center("Vertical effective stress [kPa]")

We can now calculate the settlement in each layer using the ```applyfunction``` method with the correct column mappings for strip, circular and rectangular footing.

In [ ]:
sp.applyfunction(
    function=primaryconsolidationsettlement_nc,
    outputkey="delta z strip [m]", resultkey="delta z [m]",
    parametermapping={
        'initial_height': "Layer thickness [m]",
        'initial_voidratio': "Void ratio [-]",
        "initial_effective_stress": "Vertical effective stress center [kPa]",
        "effective_stress_increase": "delta sigma z strip [kPa]",
        "compression_index": "Cc [-]"
    })
sp.applyfunction(
    function=primaryconsolidationsettlement_nc,
    outputkey="delta z circle [m]", resultkey="delta z [m]",
    parametermapping={
        'initial_height': "Layer thickness [m]",
        'initial_voidratio': "Void ratio [-]",
        "initial_effective_stress": "Vertical effective stress center [kPa]",
        "effective_stress_increase": "delta sigma z circle [kPa]",
        "compression_index": "Cc [-]"
    })
sp.applyfunction(
    function=primaryconsolidationsettlement_nc,
    outputkey="delta z rectangle [m]", resultkey="delta z [m]",
    parametermapping={
        'initial_height': "Layer thickness [m]",
        'initial_voidratio': "Void ratio [-]",
        "initial_effective_stress": "Vertical effective stress center [kPa]",
        "effective_stress_increase": "delta sigma z rectangle [kPa]",
        "compression_index": "Cc [-]"
    })
sp

Finally, the settlement can be obtained as the sum of the settlements over all the layers:

In [ ]:
settlement_strip = sp["delta z strip [m]"].sum()
print("The settlement under the strip footing is %.3fm." % settlement_strip)
settlement_circle = sp["delta z circle [m]"].sum()
print("The settlement under the circular footing is %.3fm." % settlement_circle)
settlement_rectangle = sp["delta z rectangle [m]"].sum()
print("The settlement under the square footing is %.3fm." % settlement_rectangle)

The results show that adopting the calculation for a strip footing would be too conservative when dealing with footings of finite length.